In [2]:
using LinearAlgebra

function lu_decomposition(matrix)
    n = size(matrix, 1)
    lu_matrix = copy(matrix)

    for k in 1:n
        for i in k+1:n
            factor = lu_matrix[i, k] / lu_matrix[k, k]
            lu_matrix[i, k] = factor
            lu_matrix[i, k+1:n] -= factor * lu_matrix[k, k+1:n]
        end
    end

    return lu_matrix
end

lu_decomposition (generic function with 1 method)

In [3]:
using PrettyTables

A = [
    1.0 -2.0 1.0;
    2.0 1.0 -3.0;
    4.0 -7.0 1.0
]

b = [0; 5; -1]
LU = lu_decomposition(A)

println("LU matrix:")
pretty_table(LU)

LU matrix:
┌────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │
├────────┼────────┼────────┤
│    1.0 │   -2.0 │    1.0 │
│    2.0 │    5.0 │   -5.0 │
│    4.0 │    0.2 │   -2.0 │
└────────┴────────┴────────┘


In [4]:
function lu_decomposition_with_pivot(matrix)
    n = size(matrix, 1)
    lu_matrix = copy(matrix)
    pivot = collect(1:n)
    
    for k in 1:n
        # Find the pivot row
        pivot_row = argmax(abs.(lu_matrix[k:n, k])) + k - 1

        # Swap rows in Lu_matrix and update the pivot array
        lu_matrix[[k, pivot_row], :] = lu_matrix[[pivot_row k], :]
        pivot[k], pivot[pivot_row] = pivot[pivot_row], pivot[k]

        for i in k+1:n
            factor = lu_matrix[i, k] / lu_matrix[k, k]
            lu_matrix[i, k] = factor
            lu_matrix[i, k+1:n] -= factor * lu_matrix[k, k+1:n]
        end
    end

    return lu_matrix, pivot
end

lu_decomposition_with_pivot (generic function with 1 method)

In [5]:
A = [
    1.0 -2.0 1.0;
    2.0 1.0 -3.0;
    4.0 -7.0 1.0
]

b = [0; 5; -1]
LU_wp, P = lu_decomposition_with_pivot(A)

println("LU matrix")
pretty_table(LU_wp)
println("P matrix:")
pretty_table(P)

LU matrix
┌────────┬────────────┬──────────┐
│ Col. 1 │     Col. 2 │   Col. 3 │
├────────┼────────────┼──────────┤
│    4.0 │       -7.0 │      1.0 │
│    0.5 │        4.5 │     -3.5 │
│   0.25 │ -0.0555556 │ 0.555556 │
└────────┴────────────┴──────────┘
P matrix:
┌────────┐
│ Col. 1 │
├────────┤
│      3 │
│      2 │
│      1 │
└────────┘


In [6]:
function inverse_using_compact_lu(LU, pivot)
    n = size(LU, 1)
    I_matrix = Matrix{Float64}(I, n, n)
    pivot_inv = copy(pivot)

    for (index, epivot) in pairs(pivot_inv)
        pivot_inv[index] = n - pivot[index] + 1
    end

    LU_inv = I_matrix[ :, pivot ]

    for j in 1:n
        # Solve Ly = I for y
        y = I_matrix[:, j]
        for i in 1:n
            for k in 1:i-1
                y[i] -= LU[i, k] * y[k]
            end
        end

        # Solve Ux = y for x
        x = zeros(n)
        for i in n:-1:1
            for k in i+1:n
                y[i] -= LU[i, k] * x[k]
            end
            x[i] = y[i] / LU[i, i]
        end

        # Update the inverse matrix column with the solution x
        LU_inv[:, j] = x
    end

    LU_inv = reverse(LU_inv[ :, pivot_inv], dims=2)

    return LU_inv
end

inverse_using_compact_lu (generic function with 1 method)

In [7]:
Ainv = inverse_using_compact_lu(LU_wp, P)
pretty_table(Ainv)

┌────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │
├────────┼────────┼────────┤
│    2.0 │    0.5 │   -0.5 │
│    1.4 │    0.3 │   -0.5 │
│    1.8 │    0.1 │   -0.5 │
└────────┴────────┴────────┘


In [8]:
function thomas_algorithm(coeff_matrix, rhs)
    n = size(coeff_matrix, 1)

    # Forward elimination
    for i in 2:n
        m = coeff_matrix[i, i-1] / coeff_matrix[i-1, i-1]
        coeff_matrix[i, i] -= m * coeff_matrix[i-1, i]
        rhs[i] -= m * rhs[i-1]
    end

    # Backward substitution
    x = zeros(eltype(coeff_matrix), n)
    x[n] = rhs[n] / coeff_matrix[n, n]

    for i in n-1:-1:1
        x[i] = (rhs[i] - coeff_matrix[i, i+1] * x[i+1]) / coeff_matrix[i, i]
    end

    return x
end

thomas_algorithm (generic function with 1 method)

In [9]:
coeff_matrix = [
    4.0 1.0 0.0 0.0 0.0 0.0;
    2.0 5.0 2.0 0.0 0.0 0.0;
    0.0 3.0 6.0 3.0 0.0 0.0;
    0.0 0.0 4.0 7.0 4.0 0.0;
    0.0 0.0 0.0 5.0 8.0 5.0;
    0.0 0.0 0.0 0.0 6.0 9.0
]

rhs = [11.0, 12.0, 13.0, 14.0, 15.0, 16.0]

x = thomas_algorithm(coeff_matrix, rhs)

println("Solution x:")
pretty_table(x)

Solution x:
┌──────────┐
│   Col. 1 │
├──────────┤
│  1.92857 │
│  3.28571 │
│ -4.14286 │
│  9.33333 │
│ -8.69048 │
│  7.57143 │
└──────────┘


In [10]:
function gauss_seidel(A, b, x0=zeros(size(b)); imax=100, es=1e-6)
    n = length(b)
    x = copy(x0)

    for iter in 1:imax
        for i in 1:n
            sigma = 0.0
            for j in 1:n
                if j != i
                    sigma += A[i, j] * x[j]
                end
            end
            x[i] = (b[i] - sigma) / A[i, i]
        end

        # Check for convergence
        if norm(A * x - b) < es
            println("Converged after $iter iterations.")
            return x
        end
    end

    println("Did not converge after $imax iterations.")
    return x
end

gauss_seidel (generic function with 2 methods)

In [11]:
A = [
    4.0 -1.0 2.0;
    1.0 5.0 1.0;
    2.0 1.0 3.0
]

b = [8.0, 7.0, 9.0]

x = gauss_seidel(A, b)

println("Solution x:")
pretty_table(x)

Converged after 17 iterations.
Solution x:
┌──────────┐
│   Col. 1 │
├──────────┤
│  1.23077 │
│ 0.769231 │
│  1.92308 │
└──────────┘


# Task

In [12]:
A = [
    2.0 1.0 -1.0;
    -3.0 -1.0 2.0;
    -2.0 1.0 2.0
]

LU_matrix, Pivot = lu_decomposition_with_pivot(A)
A_inverse = inverse_using_compact_lu(LU_matrix, Pivot)
println("Inverse Matrix A:")
pretty_table(A_inverse)

Inverse Matrix A:
┌────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │
├────────┼────────┼────────┤
│    4.0 │    3.0 │   -1.0 │
│   -2.0 │   -2.0 │    1.0 │
│    5.0 │    4.0 │   -1.0 │
└────────┴────────┴────────┘
